# Dart-Throwing

Final Push TODOs:

- visualize dart path with SetLineSegments() (both projected and tracked actual)
- increase wsg force limit, dart mu

Paper:
- talk about things we tried:
    - linear ramp-in/ramp-out (in cartesian)
        - problems with exiting workspace limits
    - "perturbed" before and after poses so we reach the correct velocity
    - RRT* for ramp up into poses near workspace limit (in joint coords)
    - finally: linear ramp-in/ramp-out in joint coords
        - joint limits, collision limits

In [ ]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt, mpld3
from IPython.display import HTML, display, SVG
import pydot
from os.path import exists

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, MeshcatVisualizerCpp, 
    ConstantVectorSource, MeshcatVisualizerParams, DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePose, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource,
    PassThrough, Demultiplexer, InverseDynamicsController, Adder, StateInterpolatorWithDiscreteDerivative,
    SchunkWsgPositionController, MakeMultibodyStateToWsgStateSystem,
    MathematicalProgram, Solve, eq, le, ge, AutoDiffXd, PiecewiseQuaternionSlerp, PiecewisePose, PiecewisePolynomial, 
    Capsule, Cylinder, CoulombFriction, InitializeAutoDiff, 
    AbstractValue, BasicVector, ExternallyAppliedSpatialForce, AbstractValues, SpatialForce
)
from pydrake.examples.manipulation_station import ManipulationStation
from manipulation.meshcat_cpp_utils import StartMeshcat, AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.scenarios import AddMultibodyTriad
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import Problem
from manipulation.exercises.trajectories.rrt_planner.robot import Range, ConfigurationSpace

from manipulation.scenarios import AddIiwa, AddWsg, AddRgbdSensors
from manipulation.utils import FindResource


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

Meshcat is now available at https://865a3533-96c6-49e9-ab4c-53138efbbf27.deepnoteproject.com


## Drake Plant/System Setup

In [ ]:
dartboard_pose = RigidTransform(RollPitchYaw(0, 0, np.pi/2), np.array([1.5, -0.3, 0.5]))
wsg_position_open = 0.107
wsg_position_close = 0.001

In [ ]:
class DartAerodynamics(LeafSystem):
    """
    Manages dart aerodynamics
    """
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self.p_Bcm = [-2.1263e-09,-0.10,-1.4616e-08] # np.zeros(3)
        self._plant = plant
        self._body = self._plant.GetBodyByName("dart")
        self.best_distance = float('inf')

        self.drag_matrix = np.array([[1e-4,   0,   0,   0,   0,   0],
                                      [0,   0,   0,   0,   0,   0],
                                      [0,   0,   1e-4,   0,   0,   0],
                                      [0,   0,   0,   3.5e-3, 0,   0],
                                      [0,   0,   0,   0,   3.3e-6, 0],
                                      [0,   0,   0,   0,   0, 3.5e-3]])

        # self.drag_matrix = np.zeros((6,6))

        self.dart_state_port = self.DeclareVectorInputPort("dart_state", BasicVector(13))
        self.DeclareAbstractOutputPort(
            "spatial_forces_vector", lambda: AbstractValue.Make(
            [ExternallyAppliedSpatialForce()]), self.CalcForces)
    
    def CalcForces(self, context, output):
        dart_state = self.dart_state_port.Eval(context)
        dart_pos = dart_state[0:7]
        dart_vel = dart_state[7:13]
        dart_vel = np.expand_dims(dart_vel, axis=0)
        dart_vel = dart_vel.T
        distance_from_board = np.linalg.norm(dart_pos[4:7] - np.array([1.5, -0.3, 0.5]))
        if distance_from_board < self.best_distance:
            self.best_distance = distance_from_board
            print(self.best_distance)

        force = ExternallyAppliedSpatialForce()
        force.body_index = self._body.index()
        X_WD = RigidTransform(Quaternion(dart_pos[0:4]/np.linalg.norm(dart_pos[0:4])),[0,0,0])
        v_dart_D = np.zeros(dart_vel.shape)
        v_dart_D[0:3] = X_WD.inverse().multiply(dart_vel[0:3])
        v_dart_D[3:6] = X_WD.inverse().multiply(dart_vel[3:6])
        f_dart_D = -self.drag_matrix @ (np.power(v_dart_D, 2))
        f_dart_D = f_dart_D * np.sign(v_dart_D)
        # print(f_dart_D)
        force.p_BoBq_B = self.p_Bcm
        f_dart_W_f = X_WD.multiply(f_dart_D[3:6])
        f_dart_W_tau = X_WD.multiply(f_dart_D[0:3])
        # print("x wd ", X_WD)
        # print("inverse ", X_WD.inverse())
        # print(f"force: {f_dart_W_f}")
        # print(f"torque: {f_dart_W_tau}")
        force.F_Bq_W = SpatialForce(tau=f_dart_W_tau, f=f_dart_W_f)
        output.set_value([force])
        

# class PseudoInverseController(LeafSystem):
#     """
#     From Robot Painter Example
#     """
#     def __init__(self, plant):
#         LeafSystem.__init__(self)
#         self._plant = plant
#         self._plant_context = plant.CreateDefaultContext()
#         self._iiwa = plant.GetModelInstanceByName("iiwa7")
#         self._G = plant.GetBodyByName("body").body_frame()
#         self._W = plant.world_frame()

#         self.w_G_port = self.DeclareVectorInputPort("w_G", BasicVector(3))
#         self.v_G_port = self.DeclareVectorInputPort("v_G", BasicVector(3))
#         self.q_port = self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
#         self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7), 
#                                      self.CalcOutput)
#         self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
#         self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

#     def CalcOutput(self, context, output):
#         w_G = self.w_G_port.Eval(context)
#         v_G = self.v_G_port.Eval(context)
#         V_G = np.hstack([w_G, v_G])
#         q = self.q_port.Eval(context)
#         self._plant.SetPositions(self._plant_context, self._iiwa, q)
#         J_G = self._plant.CalcJacobianSpatialVelocity(
#             self._plant_context, JacobianWrtVariable.kV, 
#             self._G, [0,0,0], self._W, self._W)
#         J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
#         v = np.linalg.pinv(J_G).dot(V_G) #important
#         output.SetFromVector(v)

def MakePhysicalSetup(dartboard_pose, time_step=0.002, weld_dart = False):
    """
        Modified version of MakeManipulationStation from the piazza post:
        https://piazza.com/class/kt3s6xjnxl84xx?cid=336

        Adds iiwa, wsg, iiwa controller, wsg controller, dart, dartboard

        Returns a diagram
    """
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=time_step)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa)
    Parser(plant).AddModelFromFile(
        FindResource("models/camera_box.sdf"), "camera0")
    Parser(plant).AddModelFromFile("dartboard_obj_color.urdf", "dartboard")
    Parser(plant).AddModelFromFile("dart.urdf", "dart") 

    plant.WeldFrames(
        frame_on_parent_P=plant.world_frame(),
        frame_on_child_C=plant.GetFrameByName("base"),
        X_PC=RigidTransform(RollPitchYaw(0, 0, np.pi/2), np.array([0.5, 0, 1.5])))

    plant.WeldFrames(
        frame_on_parent_P=plant.world_frame(),
        frame_on_child_C=plant.GetFrameByName("dartboard"),
        X_PC=RigidTransform(dartboard_pose))

    if weld_dart:
        # # # attach dart to gripper 
        plant.WeldFrames(
            frame_on_parent_P=plant.GetFrameByName('body'),
            frame_on_child_C=plant.GetFrameByName("dart"),
            X_PC=RigidTransform(RollPitchYaw(np.pi/2, np.pi, np.pi/2), np.array([0., 0.08, 0])))
        # plant.SetDefaultFreeBodyPose("dart", RigidTransform(RollPitchYaw(0, 0, np.pi/2), np.array([1.5, -0.3, 0.5])))

    # # # set collision geometries for dart and dartboard
    cylinder = Cylinder(.23, .01) # radius, length
    capsule = Cylinder(0.003, 0.08) # radius, length
    # capsule = Cylinder(0.009, 0.08)
    X_BCylinder = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), np.array([0, 0.02, 0])) # fixed pose of collision geometry in dartboard frame
    X_DCapsule = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), np.array([0, 0.0, 0])) # fixed pose of collision geometry in dart frame
    mu = 1.5
    plant.RegisterCollisionGeometry(plant.GetBodyByName('dartboard'), X_BCylinder, cylinder, "dartboard_col", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.GetBodyByName('dartboard'), X_BCylinder, cylinder, "dartboard_col", [.9, .9, .9, 1.0])
    plant.RegisterCollisionGeometry(plant.GetBodyByName('dart'), X_DCapsule, capsule, "dart_col", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.GetBodyByName('dart'), X_DCapsule, capsule, "dart_col", [.9, .9, .9, 1.0])

    plant.set_name("mb_plant")

    plant.Finalize()
    
    num_iiwa_positions = plant.num_positions(iiwa)

    # Export dartboard and dart state:
    # dartboard = plant.GetModelInstanceByName("dartboard")
    # builder.ExportOutput(plant.get_state_output_port(dartboard), "dartboard_state")
    # dart = plant.GetModelInstanceByName("dart")
    # builder.ExportOutput(plant.get_state_output_port(dart), "dart_state")

    dart = plant.GetModelInstanceByName("dart")

    dart_aerodynamics = builder.AddNamedSystem("dart_aero", DartAerodynamics(plant))
    builder.Connect(plant.get_state_output_port(dart), 
                        dart_aerodynamics.GetInputPort("dart_state"))
    builder.Connect(dart_aerodynamics.GetOutputPort("spatial_forces_vector"), 
                        plant.get_applied_spatial_force_input_port())

    # I need a PassThrough system so that I can export the input port.
    iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
    builder.ExportInput(iiwa_position.get_input_port(), "iiwa_position")
    builder.ExportOutput(iiwa_position.get_output_port(), "iiwa_position_command")

    # Export the iiwa "state" outputs.
    demux = builder.AddSystem(Demultiplexer(
        2 * num_iiwa_positions, num_iiwa_positions))
    builder.Connect(plant.get_state_output_port(iiwa), demux.get_input_port())
    builder.ExportOutput(demux.get_output_port(0), "iiwa_position_measured")
    builder.ExportOutput(demux.get_output_port(1), "iiwa_velocity_estimated")
    builder.ExportOutput(plant.get_state_output_port(iiwa), "iiwa_state_estimated")

    # Make the plant for the iiwa controller to use.
    controller_plant = MultibodyPlant(time_step=time_step)
    controller_iiwa = AddIiwa(controller_plant)
    AddWsg(controller_plant, controller_iiwa, welded=True)
    controller_plant.Finalize()

    # Add the iiwa controller
    iiwa_controller = builder.AddSystem(
        InverseDynamicsController(
            controller_plant,
            kp=[100]*num_iiwa_positions,
            ki=[1]*num_iiwa_positions,
            kd=[20]*num_iiwa_positions,
            has_reference_acceleration=False))
    iiwa_controller.set_name("iiwa_controller")
    builder.Connect(
        plant.get_state_output_port(iiwa), iiwa_controller.get_input_port_estimated_state())

    # Add in the feed-forward torque
    adder = builder.AddSystem(Adder(2, num_iiwa_positions))
    builder.Connect(iiwa_controller.get_output_port_control(),
                    adder.get_input_port(0))
    # Use a PassThrough to make the port optional (it will provide zero values if not connected).
    torque_passthrough = builder.AddSystem(PassThrough([0]*num_iiwa_positions))
    builder.Connect(torque_passthrough.get_output_port(),
                    adder.get_input_port(1))
    builder.ExportInput(torque_passthrough.get_input_port(), 
                        "iiwa_feedforward_torque")
    builder.Connect(adder.get_output_port(),
                    plant.get_actuation_input_port(iiwa))

    # Add discrete derivative to command velocities.
    desired_state_from_position = builder.AddSystem(
        StateInterpolatorWithDiscreteDerivative(
            num_iiwa_positions, time_step, suppress_initial_transient=True))
    desired_state_from_position.set_name("desired_state_from_position")
    builder.Connect(desired_state_from_position.get_output_port(),      
                    iiwa_controller.get_input_port_desired_state())
    builder.Connect(iiwa_position.get_output_port(), 
                    desired_state_from_position.get_input_port())

    # Export commanded torques.
    #builder.ExportOutput(adder.get_output_port(), "iiwa_torque_commanded")
    #builder.ExportOutput(adder.get_output_port(), "iiwa_torque_measured")

    # Wsg controller.
    wsg_controller = builder.AddSystem(SchunkWsgPositionController())
    wsg_controller.set_name("wsg_controller")
    builder.Connect(
        wsg_controller.get_generalized_force_output_port(),             
        plant.get_actuation_input_port(wsg))
    builder.Connect(plant.get_state_output_port(wsg),
                    wsg_controller.get_state_input_port())
    builder.ExportInput(wsg_controller.get_desired_position_input_port(), 
                        "wsg_position")
    builder.ExportInput(wsg_controller.get_force_limit_input_port(),  
                        "wsg_force_limit")
    wsg_mbp_state_to_wsg_state = builder.AddSystem(
        MakeMultibodyStateToWsgStateSystem())
    builder.Connect(plant.get_state_output_port(wsg), 
                    wsg_mbp_state_to_wsg_state.get_input_port())
    builder.ExportOutput(wsg_mbp_state_to_wsg_state.get_output_port(), 
                         "wsg_state_measured")
    builder.ExportOutput(wsg_controller.get_grip_force_output_port(), 
                         "wsg_force_measured")

    # Cameras.
    AddRgbdSensors(builder, plant, scene_graph)

    # Export "cheat" ports.
    builder.ExportOutput(scene_graph.get_query_output_port(), "geometry_query")
    builder.ExportOutput(plant.get_contact_results_output_port(), 
                         "contact_results")
    builder.ExportOutput(plant.get_state_output_port(), 
                         "plant_continuous_state")

    diagram = builder.Build()
    return diagram, scene_graph

In [ ]:
"""
Greg 11/27: tried to run the stuff that you had in the cell below, but some stuff didn't work
I read through some documentation and i think i understand what is going on now??

MultibodyPlant (System): represents a bunch of physical objects, can have MULTIPLE objects
    - SetPositionsAndVelocities(), SetPositions(), Finalize()

Diagram (System): how to stick systems together. edges are inputs -> outputs, vertices are systems

DiagramBuilder: makes diagrams, when you are done connecting stuff together, call
Build() (returns a diagram). 
    - AddSystem() returns a reference to the system you just added


The ManipulationStation example makes a diagram (which is also a system) with:
    - MultibodyPlant (iiwa, WSG) + dartboard, dart <- the only one we should need
    - Some systems for the iiwa and WSG controller
    - NOTE: iiwa states are discrete states not continous states
        - 11/27: confirmed that setting the states works!!!

It has output ports: 
    - iiwa_position_command, iiwa_position_measured, iiwa_velocity_estimated, iiwa_state_estimated
        iiwa_torque_commanded, iiwa_torque_measured, wsg_state_measured, wsg_force_measured,
        geometry_query, contact_results, plant_continuous_state

And inputs:
    - iiwa_position, iiwa_feedforward_torque, wsg_position, wsg_force_limit

We then want to make another diagram that has:
    - ManipulationStation
    - System that feeds ManipulationStation a time series of position commands
"""

class IIWA_Thrower():
    def __init__(self, dartboard_pose, vis=True, start_state = None, time_step=0.002, traj=None, wsg_traj=None, weld_dart = False):
        self.start_state = start_state
        self.vis = vis
        self.weld_dart = weld_dart

        builder = DiagramBuilder()
        self.phys_setup, self.scene_graph = MakePhysicalSetup(dartboard_pose, weld_dart=self.weld_dart)
        self.phys_setup = builder.AddNamedSystem("phys_setup", self.phys_setup)

        # display(SVG(pydot.graph_from_dot_data(self.phys_setup.GetGraphvizString())[0].create_svg()))

        self.plant = self.phys_setup.GetSubsystemByName("mb_plant")

        # optionally add trajectory source
        if traj is not None:
            joint_position = builder.AddSystem(TrajectorySource(traj))
            builder.Connect(joint_position.get_output_port(),
                            self.phys_setup.GetInputPort("iiwa_position"))

        if self.vis:
            params = MeshcatVisualizerParams()
            params.delete_on_initialization_event = False
            self.visualizer = MeshcatVisualizerCpp.AddToBuilder(
                builder, self.phys_setup.GetOutputPort("geometry_query"), meshcat, params)
        
        if wsg_traj is not None:
            wsg_position = builder.AddSystem(TrajectorySource(wsg_traj))
            builder.Connect(wsg_position.get_output_port(), self.phys_setup.GetInputPort("wsg_position"))
        else:
            wsg_position = builder.AddSystem(ConstantVectorSource([wsg_position_close]))
            builder.Connect(wsg_position.get_output_port(), self.phys_setup.GetInputPort("wsg_position"))
        wsg_force = builder.AddSystem(ConstantVectorSource([1e4]))
        builder.Connect(wsg_force.get_output_port(), self.phys_setup.GetInputPort("wsg_force_limit"))

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.dart_frame = self.plant.GetFrameByName('dart')
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext(start_state)

        self.diagram.Publish(context)
    
    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies
        
        Input: 
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.
        
        Frames whose names already exist will be overwritten by the new frame
        """
        if self.vis:
            AddMeshcatTriad(meshcat, "thrower/" + name,
                            length=length, radius=radius, X_PT=X_WF)

    def CreateDefaultContext(self, q0=None):
        context = self.diagram.CreateDefaultContext()

        if q0 is None:
            q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05, 
                    -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        
        if not self.weld_dart:
            self.set_iiwa_and_gripper(q0, 0.004, context)
            # place dart in between gripper fingers
            plant_context = self.diagram.GetSubsystemContext(self.plant,context)
            dart = self.plant.GetBodyByName("dart")
            X_WG = self.plant.CalcRelativeTransform(
                        plant_context,
                        frame_A=self.world_frame,
                        frame_B=self.gripper_frame)
            X_WD = X_WG.multiply(RigidTransform(RollPitchYaw(np.pi/2, np.pi, np.pi/2), np.array([0., 0.08, 0])))
            X_WD = X_WG.multiply(RigidTransform(RollPitchYaw(-np.pi/2, np.pi, np.pi/2), np.array([0., 0.08, 0])))
            self.plant.SetFreeBodyPose(plant_context, dart, X_WD)
            self.visualize_frame("dart", X_WD)
            print(dart.CalcCenterOfMassInBodyFrame(context))
        else:
            self.set_iiwa_and_gripper(q0, wsg_position_open, context)


        return context

    def set_iiwa_and_gripper(self, q_iiwa, gripper_setpoint, context):
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, context)
        phys_setup_context = self.diagram.GetMutableSubsystemContext(self.phys_setup, context)

        mb_plant = self.phys_setup.GetSubsystemByName("mb_plant")
        mb_plant_context = self.phys_setup.GetMutableSubsystemContext(mb_plant, phys_setup_context)

        iiwa = mb_plant.GetModelInstanceByName("iiwa7")
        mb_plant.SetPositions(mb_plant_context, iiwa, q_iiwa)

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), 
                q_iiwa)

        q_gripper = [-gripper_setpoint/2, gripper_setpoint/2]

        wsg = mb_plant.GetModelInstanceByName("Schunk_Gripper")
        mb_plant.SetPositions(mb_plant_context, wsg, q_gripper)


    def throw(self, sim_speed=1.0, time = 1.0):
        context = self.CreateDefaultContext(self.start_state)
        plant_context = self.diagram.GetSubsystemContext(self.plant,context)
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(sim_speed)

        iiwa = self.plant.GetModelInstanceByName("iiwa7")

        times = []
        dart_poses = []
        gripper_poses = []
        iiwa_state = []

        # duration = time if running_as_notebook else 0.01
        # simulator.AdvanceTo(duration)



        
        for t in np.arange(0, time, 0.01):
            simulator.AdvanceTo(t)
            times.append(t)
            X_WG = self.plant.CalcRelativeTransform(
                        plant_context,
                        frame_A=self.world_frame,
                        frame_B=self.gripper_frame)
            X_WD = self.plant.CalcRelativeTransform(
                        plant_context,
                        frame_A=self.world_frame,
                        frame_B=self.dart_frame)
            gripper_poses.append(X_WG)
            dart_poses.append(X_WD)
            iiwa_state.append(self.plant.GetPositionsAndVelocities(plant_context, iiwa))
        

        return times, dart_poses, gripper_poses, iiwa_state


    def ExistsCollision(self, q_iiwa, gripper_setpoint):
            """
            :param q_iiwa: (7,) numpy array, joint angle of robots in radian.
            :param gripper_setpoint: float, gripper opening distance in meters.
            """
            context_diagram = self.CreateDefaultContext()
            context_plant = self.diagram.GetSubsystemContext(
                self.plant, context_diagram)
            context_scene_graph = self.diagram.GetSubsystemContext(
                 self.scene_graph, context_diagram)

            self.set_iiwa_and_gripper(q_iiwa, gripper_setpoint, context_diagram)

            query_object = self.scene_graph.GetOutputPort("query").Eval(context_scene_graph)
            collision_pairs = query_object.ComputePointPairPenetration()

            if len(collision_pairs) > 0:
                for pair in collision_pairs:
                    print(f"idA: {pair.id_A}, idB: {pair.id_B}")

            return len(collision_pairs) > 0

In [ ]:
meshcat.Delete()

thrower = IIWA_Thrower(dartboard_pose)

[-2.1263e-09  2.0000e-02 -1.4616e-08]


## Dart Release Pose/Velocity Optimization

In [ ]:
def get_iiwa_ee_gripper_pose(plant_f, q):
    """
    Custom evaluator that can handle both float and AutoDiffXd inputs
    https://github.com/RobotLocomotion/drake/blob/master/tutorials/mathematical_program_multibody_plant.ipynb
    """
    if q.dtype == float:
        plant = plant_f
    else:
        # Assume AutoDiff.
        plant = plant_f.ToAutoDiffXd()
    context = plant.CreateDefaultContext()

    gripper_frame = plant.GetFrameByName('body')
    world_frame = plant.world_frame()

    context = plant.CreateDefaultContext()
    iiwa = plant.GetModelInstanceByName("iiwa7")
    plant.SetPositions(context, iiwa, q)
    X_WE = plant.CalcRelativeTransform(
                    context,
                    frame_A=world_frame,
                    frame_B=gripper_frame)
    
    X_EG = RigidTransform(RotationMatrix(), np.array([0., 0.08, 0]))

    if q.dtype != float:
        X_EG = X_EG.cast[AutoDiffXd]()

    X_WG = X_WE.multiply(X_EG)

    return X_WE, X_WG



def get_iiwa_gripper_velocity(plant_f, state):
    """
    Custom evaluator that can handle both float and AutoDiffXd inputs
    https://github.com/RobotLocomotion/drake/blob/master/tutorials/mathematical_program_multibody_plant.ipynb

    Returns spatial velocities (spatial vector docs: https://drake.mit.edu/doxygen_cxx/group__multibody__spatial__vectors.html)
    """

    q = state[0:7]
    v = state[7:14]

    if q.dtype == float:
        plant = plant_f
    else:
        # Assume AutoDiff.
        plant = plant_f.ToAutoDiffXd()
    context = plant.CreateDefaultContext()

    gripper_frame = plant.GetFrameByName('body')
    world_frame = plant.world_frame()

    context = plant.CreateDefaultContext()
    iiwa = plant.GetModelInstanceByName("iiwa7")
    plant.SetPositions(context, iiwa, q)

    J_G_spatial = plant.CalcJacobianSpatialVelocity(
        context, JacobianWrtVariable.kQDot, 
        gripper_frame, np.array([0., 0.08, 0]), world_frame, world_frame)

    J_G_spatial = J_G_spatial[:,0:7] # Ignore gripper terms

    gripper_cartesian_spatial_v = J_G_spatial @ v

    return gripper_cartesian_spatial_v

In [ ]:
"""
Inverse Dart Kinematics
"""

def dart_flight_location_constraint(X_WB, plant, state):
    """
        Wrapper to use dart_flight position as a constraint.
    """

    q = state[0:7]
    v = state[7:14]

    _, X_WG = get_iiwa_ee_gripper_pose(plant, q)

    X_WG_position = X_WG.translation()
    
    v_G_3_6 = get_iiwa_gripper_velocity(plant, state)[3:6]

    t, vp_WD_final, p_WD_final = dart_flight(X_WB, X_WG_position, v_G_3_6)

    if p_WD_final is None:
        return [t]

    displacement = X_WB.translation() - p_WD_final
    distance = (displacement[0]**2 + displacement[1]**2 + displacement[2]**2)**(1/2)

    return [distance]

def dart_flight_time_cost(X_WB, plant, state):
    """
        Wrapper to use dart_flight time as a cost.
    """

    q = state[0:7]
    v = state[7:14]

    _, X_WG = get_iiwa_ee_gripper_pose(plant, q)
    X_WG_position = X_WG.translation()
    v_G_3_6 = get_iiwa_gripper_velocity(plant, state)[3:6]

    t, vp_WD_final, p_WD_final = dart_flight(X_WB, X_WG_position, v_G_3_6)

    return t

def dart_flight(X_WB, p_WG, vp_G):
    """
        Assume dartboard is along positive x direction from world frame, on a wall with
        the wall normal along the x direction.

        Ignores aerodynamics of the dart.

        X_WB: Pose of dartboard in relation to World frame
        p_WG: position of gripper in relation to World frame
        vp_G: translational velocity of gripper
        
        Returns: duration of dart flight, RigidTransform of final dart pose in world frame
    """

    # Gravitational acceleration of dart
    a_W = np.array([0,0,-9.81])

    # Assuming the dartboard is on a wall, where the wall normal is in the x direction
    # we can find the time that the dart hits the wall using the x velocity/positions
    t = (X_WB.translation()[0] - p_WG[0])/vp_G[0]

    # Final position and velocity of the dart, assuming the dart is always "inline"
    # with its velocity vector, the velocity vector can give us the orientation
    p_WD_final = p_WG + t*vp_G + 1/2*np.power(t, 2)*a_W
    vp_WD_final = vp_G + a_W*t

    # print(f"t:{t}, p_WD_final:{p_WD_final}, vp_WD_final:{vp_WD_final}")

    return t, vp_WD_final, p_WD_final

def find_release_point(plant, X_WB, v_max, p_WBB_small, p_WBB_large, tolerance):
    """
    Inputs:
        X_WB: Dartboard pose in world frame
        f_G: forward kinematics function: takes joint angles and gives pose of end effector
        fJ_G: function that takes joint angles and returns the Jacobian 
        v_max: max joint velocities (magnitudes)
        p_WBB_small: lower limit for acceptable end end effector pose
        p_WBB_large: upper limit for acceptable end end effector pose
        tolerance: acceptable distance from the end dart pose to the center of the target
    """

    # Set up MathematicalProgram
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, 'v')
    q = prog.NewContinuousVariables(7, 'q')

    time_cost = lambda state: dart_flight_time_cost(X_WB, plant, state)**2

    # Add costs
    prog.AddCost(v.dot(v))
    # prog.AddCost(time_cost, vars=np.concatenate((q,v)))

    v_G_angular_cost = lambda state: get_iiwa_gripper_velocity(plant, state)[0:3].dot(get_iiwa_gripper_velocity(plant, state)[0:3])
    # prog.AddCost(v_G_angular_cost, vars=np.concatenate((q,v)))

    # Add Constraints

    # Joint angles should be within joint limits
    theta_max_joints = np.array([2.96706, 2.0944, 2.96706, 2.0944, 2.96706, 2.0944, 3.05433]) - 0.1
    prog.AddBoundingBoxConstraint(-theta_max_joints, theta_max_joints, q)

    # A solution with a negative time is not a solution
    time_constraint = lambda state: [dart_flight_time_cost(X_WB, plant, state)]
    prog.AddConstraint(time_constraint, lb=[0], ub=[10], vars=np.concatenate((q,v)))

    # Respect joint velocity limits
    v_max_joints = np.ones(7)*v_max
    prog.AddBoundingBoxConstraint(-v_max_joints, v_max_joints, v)

    # There should be no angular velocity
    ang_v_tolerance = 0.5# 1e-2
    v_G_angular_constraint = lambda state: get_iiwa_gripper_velocity(plant, state)[0:3]
    prog.AddConstraint(v_G_angular_constraint, lb=-ang_v_tolerance*np.ones(3), ub=ang_v_tolerance*np.ones(3), vars=np.concatenate((q,v)))

    # Translational velocity should be inline with z axis of gripper
    tv_tolerance = 1e-2 # can solve with no ang velocity constraint and tv_tolerance=.01
    v_G_tvel_constraint_x = lambda state: [(get_iiwa_ee_gripper_pose(plant, state[0:7]))[0].rotation().multiply([1,0,0]).dot(get_iiwa_gripper_velocity(plant, state)[3:6])]
    v_G_tvel_constraint_y = lambda state: [(get_iiwa_ee_gripper_pose(plant, state[0:7]))[0].rotation().multiply([0,1,0]).dot(get_iiwa_gripper_velocity(plant, state)[3:6])]
    v_G_tvel_constraint_z = lambda state: [(get_iiwa_ee_gripper_pose(plant, state[0:7]))[0].rotation().multiply([0,0,1]).dot(get_iiwa_gripper_velocity(plant, state)[3:6])]
    prog.AddConstraint(v_G_tvel_constraint_x, lb=[-tv_tolerance], ub=[tv_tolerance], vars=np.concatenate((q,v)))
    prog.AddConstraint(v_G_tvel_constraint_y, lb=[-tv_tolerance], ub=[tv_tolerance], vars=np.concatenate((q,v)))
    prog.AddConstraint(v_G_tvel_constraint_z, lb=[0], ub=[1e5], vars=np.concatenate((q,v)))

    # Pose needs to be in bounding box
    X_WG_translation_constraint = lambda q: (get_iiwa_ee_gripper_pose(plant, q))[0].translation()
    prog.AddConstraint(X_WG_translation_constraint, lb=p_WBB_small, ub=p_WBB_large, vars=q)

    # Dart needs to hit bullseye of target
    dart_location_constraint = lambda state: dart_flight_location_constraint(X_WB, plant, state)
    prog.AddConstraint(dart_location_constraint, lb=[0], ub=[tolerance], vars=np.concatenate((q,v)))

    # Solve

    while True:
        q_init = np.min(theta_max_joints)*np.random.rand(7)
        v_init = 2*v_max*np.random.rand(7)-v_max*np.ones(7)
        prog.SetInitialGuess(q, q_init)
        prog.SetInitialGuess(v, v_init)

        print(f"IC: q: {q_init} \n v: {v_init}")
        result = Solve(prog)

        if (result.is_success()): 
            break

    v_sol = result.GetSolution(v)
    q_sol = result.GetSolution(q)

    return q_sol, v_sol

In [ ]:
plant_f = MultibodyPlant(time_step=0.002)
iiwa = AddIiwa(plant_f)
AddWsg(plant_f, iiwa, welded=True)
plant_f.Finalize()

context_f = plant_f.CreateDefaultContext()
plant_ad = plant_f.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()

v_max = 5.0

p_WBB_small = np.array([0.125, -1.5, 0.4])
p_WBB_large = np.array([0.5, 0, 0.875])

thrower.visualize_frame('PBB_small', RigidTransform(RotationMatrix(), p_WBB_small), length=0.5)
thrower.visualize_frame('PBB_large', RigidTransform(RotationMatrix.MakeYRotation(np.pi).multiply(RotationMatrix.MakeZRotation(-np.pi/2)), p_WBB_large), length=0.5)

q, v = find_release_point(plant_f, dartboard_pose, v_max, p_WBB_small, p_WBB_large, 0.05)

print("------------RESULT------------")

print(f"q: {q}")
print(f"v: {v}")

X_WE_release, X_WG_release = get_iiwa_ee_gripper_pose(plant_f, q)
print(X_WG_release)
v_G_release = get_iiwa_gripper_velocity(plant_f, np.concatenate((q,v)))
print(v_G_release)

t, vp_WD_final, p_WD_final = dart_flight(dartboard_pose, X_WG_release.translation(), v_G_release[3:6])

vp_WD_final_norm = vp_WD_final/np.linalg.norm(vp_WD_final)

perp1 = np.cross(vp_WD_final, [0,0,1])
perp2 = np.cross(vp_WD_final, perp1)

perp1 = perp1/np.linalg.norm(perp1)
perp2 = perp2/np.linalg.norm(perp2)

R = np.stack([perp2/10, perp1/10, vp_WD_final_norm], axis=1)

if np.dot(R[:,2], dartboard_pose.rotation().multiply([0,0,1])) > 0:
    R[:,2] = -R[:,2]

if np.linalg.det(R) < 0:
    R[:,0] = -R[:,0]

print(f"Flight Time: {t}")

thrower.visualize_frame('dart_pos', RigidTransform(RotationMatrix(R), p_WD_final), length=0.15)
thrower.visualize_frame('release_pos', X_WG_release, length=0.3)

thrower = IIWA_Thrower(dartboard_pose, start_state=q)

IC: q: [0.09651406 0.72298357 0.9091396  0.76830344 1.62655504 0.8927481
 1.20566347] 
 v: [ 4.99168837  1.68855537  2.01930693  0.70667846  2.80706188 -0.94560629
  2.10327273]
------------RESULT------------
q: [-1.25581174  1.48286649  1.8167165  -0.02159925  1.95211914  1.33136728
 -1.58279444]
v: [ 3.42267748 -2.50027385  0.02741027 -0.12902175  0.02669622 -0.05856054
 -3.82788403]
RigidTransform(
  R=RotationMatrix([
    [-0.44689540823453394, -0.48317485363789076, 0.7528788447757664],
    [0.8880378539706818, -0.3412486339788274, 0.3081203331861294],
    [0.10804288044122151, 0.8062824756991732, 0.5815800077086357],
  ]),
  p=[0.12181486504375594, -0.8475716674850916, 0.6340742170557023],
)
[-0.5         0.5         0.5         2.72866453  1.10457848  2.1150875 ]
Flight Time: 0.5050767947316807
[-2.1263e-09  2.0000e-02 -1.4616e-08]


In [ ]:
"""
    Repeatedly run optimization in case we are stuck in local minima
"""
"""

best_q = None
best_v = np.ones(7)*1e5
best_v_G = np.ones(6)*1e5

while True:
    try:
        q, v = find_release_point(plant_f, dartboard_pose, v_max, p_WBB_small, p_WBB_large, 0.15)
        X_WG_release = get_pose(plant_f, q)
        v_G_release = get_gripper_velocity(plant_f, np.concatenate((q,v)))

        if v @ v < best_v @ best_v:
            best_v = v
            best_q = q
            best_v_G = v_G_release
            print("----------")
            print(f"Better: v: {best_v} \n q: {best_q} \n v_G: {best_v_G}")
            print(X_WG_release)
            print(f"SCORE: {v @ v}")
            print("----------")
    except KeyboardInterrupt:
        print("----------")
        print("ending")
        print(f"Best found: v: {best_v} \n q: {best_q} \n v_G: {best_v_G}")
        print(X_WG_release)
        q = best_q
        v = best_v
        print("----------")
        break

X_WG_release = get_pose(plant_f, q)
print(X_WG_release)
v_G_release = get_gripper_velocity(plant_f, np.concatenate((q,v)))
print(v_G_release)
""";

## Create iiwa/WSG trajectories (OLD

In [ ]:
def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame('frame_{}'.format(i), pose, length=0.15)

"""
start_q = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05, 
                    -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])

_, start_pose = get_pose(plant_f, start_q)

def compose_throw_poses(X_WG_release, v_G_release, X_WG_start, X_WG_end=None, t_ramp = 3):
    '''
        Compose a trajectory for the arm and gripper to throw the dart given a release pose, start pose, and
        an end pose (optional)
    '''

    v_G_release_translation = v_G_release[3:6]

    t_release = 3

    v_G_release_translation_norm = np.linalg.norm(v_G_release_translation)

    # calculate location of follow through pose
    dt_ft = 0.05
    X_WG_ft = RigidTransform(X_WG_release.rotation(), X_WG_release.translation() + v_G_release_translation*dt_ft)

    # dt_lineup = 0.1
    # X_WG_lineup = RigidTransform(X_WG_release.rotation(), X_WG_release.translation() - v_G_release_translation*dt_lineup)

    if X_WG_end is None:
        X_WG_end = X_WG_ft

    # Just used to generate intermediate pose orientations, time is rescaled later
    # orientation_traj = PiecewiseQuaternionSlerp()
    # orientation_traj.Append(0.0, X_WG_lineup.rotation())
    # orientation_traj.Append(1.0, X_WG_release.rotation())

    poses = []
    times = []

    poses.append(X_WG_start)
    times.append(0.0)

    # n_ramp_poses = 3
    # a = v_G_release_translation/dt_lineup

    # for i in range(1, n_ramp_poses-1):
    #     # We want constant acceleration up to the release point
    #     dt_int = dt_lineup/n_ramp_poses*i
    #     t_int = t_release-dt_lineup + dt_int
    #     X_WG_int = RigidTransform(RotationMatrix(orientation_traj.value(dt_int)), X_WG_lineup.translation() + 1/2*a*dt_int**2)
    #     poses.append(X_WG_int)
    #     times.append(t_int)

    # perturbed poses a little "before" and "after" the release pose, so that we can set the velocity
    # at the release pose
    dt = 0.01
    X_WG_release_p = RigidTransform(X_WG_release.rotation(), X_WG_release.translation() + v_G_release_translation*dt)
    X_WG_release_n = RigidTransform(X_WG_release.rotation(), X_WG_release.translation() - v_G_release_translation*dt)

    poses.append(X_WG_release_n)
    times.append(t_release-dt)

    poses.append(X_WG_release)
    times.append(t_release)
    
    poses.append(X_WG_release_p)
    times.append(t_release+dt)

    poses.append(X_WG_ft)
    times.append(5)

    poses.append(X_WG_ft)
    times.append(10)

    traj_iiwa = PiecewisePose.MakeLinear(times, poses)

    visualize_key_frames(poses)

    # Make WSG trajectory

    

    time_wsg = np.array([0, t_release-dt, t_release])
    pos_wsg = np.array([position_close, position_open, position_open])
    pos_wsg = np.expand_dims(pos_wsg, axis=0)

    traj_wsg = PiecewisePolynomial.FirstOrderHold(time_wsg, pos_wsg)

    return traj_iiwa, traj_wsg

traj_iiwa, traj_wsg = compose_throw_poses(X_WG_release, v_G_release, X_WG_start=start_pose)
"""


'\nstart_q = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05, \n                    -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])\n\n_, start_pose = get_pose(plant_f, start_q)\n\ndef compose_throw_poses(X_WG_release, v_G_release, X_WG_start, X_WG_end=None, t_ramp = 3):\n    \'\'\'\n        Compose a trajectory for the arm and gripper to throw the dart given a release pose, start pose, and\n        an end pose (optional)\n    \'\'\'\n\n    v_G_release_translation = v_G_release[3:6]\n\n    t_release = 3\n\n    v_G_release_translation_norm = np.linalg.norm(v_G_release_translation)\n\n    # calculate location of follow through pose\n    dt_ft = 0.05\n    X_WG_ft = RigidTransform(X_WG_release.rotation(), X_WG_release.translation() + v_G_release_translation*dt_ft)\n\n    # dt_lineup = 0.1\n    # X_WG_lineup = RigidTransform(X_WG_release.rotation(), X_WG_release.translation() - v_G_release_translation*dt_lineup)\n\n    if X_WG_end is None:\n        X_WG_end 

## Create iiwa/WSG trajectories (RRT*)

In [ ]:
class IiwaProblem(Problem):
    def __init__(self,
                 q_start: np.array,
                 q_goal: np.array,
                 gripper_setpoint: float,
                 is_visualizing=False):
        self.gripper_setpoint = gripper_setpoint

        # I think this has to be a separate copy
        self.collision_checker = IIWA_Thrower(dartboard_pose, vis=False)

        # Construct configuration space for IIWA.
        plant = self.collision_checker.plant
        nq = 7
        joint_limits = np.zeros((nq, 2))
        for i in range(nq):
            joint = plant.GetJointByName("iiwa_joint_%i" % (i + 1))
            joint_limits[i, 0] = joint.position_lower_limits()
            joint_limits[i, 1] = joint.position_upper_limits()

        range_list = []
        for joint_limit in joint_limits:
            range_list.append(Range(joint_limit[0], joint_limit[1]))

        # print(joint_limits)

        def l2_distance(q: tuple):
            sum = 0
            for q_i in q:
                sum += q_i ** 2
            return np.sqrt(sum)

        max_steps = nq * [np.pi / 180 * 2]  # three degrees
        cspace_iiwa = ConfigurationSpace(range_list, l2_distance, max_steps)

        # Call base class constructor.
        Problem.__init__(
            self,
            x=10,  # not used.
            y=10,  # not used.
            robot=None,  # not used.
            obstacles=None,  # not used.
            start=tuple(q_start),
            goal=tuple(q_goal),
            cspace=cspace_iiwa)

    def collide(self, configuration):
        q = np.array(configuration)
        return self.collision_checker.ExistsCollision(
                    q, self.gripper_setpoint, )
class TreeNode:
    def __init__(self, value, cost = 0, parent=None):
        self.value = value  # tuple of floats representing a configuration
        self.parent = parent  # another TreeNode
        self.children = []  # list of TreeNodes
        self.cost = cost # distance from parent to this node
        
class RRT:
    """
    RRT Tree.
    """
    def __init__(self, root: TreeNode, cspace: ConfigurationSpace):
        self.root = root  # root TreeNode
        self.cspace = cspace  # robot.ConfigurationSpace
        self.size = 1  # int length of path
        self.max_recursion = 1000  # int length of longest possible path

    def add_configuration(self, parent_node, child_value):
        distance = self.cspace.distance(parent_node.value, child_value)
        child_node = TreeNode(child_value, parent=parent_node, cost=distance)
        parent_node.children.append(child_node)
        self.size += 1
        return child_node

    def cost(self, node):
        cost_to_root = node.cost
        node = node.parent
        if node is not None:
            cost_to_root += node.cost
            node = node.parent

        return cost_to_root


    # Brute force nearest, handles general distance functions
    def nearest(self, configuration):
        """
        Finds the nearest node by distance to configuration in the
             configuration space.

        Args:
            configuration: tuple of floats representing a configuration of a
                robot

        Returns:
            closest: TreeNode. the closest node in the configuration space
                to configuration
            distance: float. distance from configuration to closest
        """
        assert self.cspace.valid_configuration(configuration)
        def recur(node, depth=0):
            closest, distance = node, self.cspace.distance(node.value, configuration)
            if depth < self.max_recursion:
                for child in node.children:
                    (child_closest, child_distance) = recur(child, depth+1)
                    if child_distance < distance:
                        closest = child_closest
                        child_distance = child_distance
            return closest, distance
        return recur(self.root)[0]

    def find_neighbors(self, configuration, radius):
        """
        Brute force fixed-radius nearest neighbors :/

        Returns:
            neighbors: Sorted of (cost(neighbor) + distance(neighbor, configuration), neighbor_node)
        """
        assert self.cspace.valid_configuration(configuration)

        neighbors = []

        def recur(node, depth=0):
            distance = self.cspace.distance(node.value, configuration)
            path_cost = distance + self.cost(node)
            if distance < radius:
                neighbors.append((path_cost, node))
            if depth < self.max_recursion:
                for child in node.children:
                    recur(child, depth+1)

        recur(self.root)
        
        return sorted(neighbors, key=lambda neighbor: neighbor[0])

class RRT_tools:
    def __init__(self, problem):
        # rrt is a tree 
        self.rrt_tree = RRT(TreeNode(problem.start), problem.cspace)
        problem.rrts = [self.rrt_tree]
        self.problem = problem

    def get_node_cost(self, node):
        return self.rrt_tree.cost(node)
        
    def find_nearest_node_in_RRT_graph(self, q_sample):
        nearest_node = self.rrt_tree.nearest(q_sample)
        return nearest_node

    def find_neighbors_in_RRT_graph(self, q_sample):
        neighbors = self.rrt_tree.find_neighbors(q_sample, 1.5)
        return neighbors
    
    def sample_node_in_configuration_space(self):
        q_sample = self.problem.cspace.sample()
        return q_sample

    def collision_free(self, q_start, q_end):
        return np.array(self.steer(q_start, q_end, max_distance=1e6) == q_end).all()

    def steer(self, q_start, q_end, max_distance = 1):
        '''
        Return the configuration farthest along on the line between q_start and q_end that 
        does not pass an obstacle and is within max_distance of q_start
        '''
        distance = self.problem.cspace.distance(q_start, q_end)

        vec_q_start = np.array(q_start)
        vec_q_end = np.array(q_end)

        if distance > max_distance:
            vec_q_end = vec_q_start + (vec_q_end-vec_q_start)/distance*max_distance

        
        samples = self.problem.safe_path(vec_q_start, vec_q_end)

        if len(samples) > 1:
            return tuple(samples[-1])
        else:
            return None
    
    def grow_rrt_tree(self, parent_node, q_sample):
        """
        add q_sample to the rrt tree as a child of the parent node
        returns the rrt tree node generated from q_sample
        """
        child_node = self.rrt_tree.add_configuration(parent_node, q_sample)
        return child_node
    
    def node_reaches_goal(self, node):
        return np.array(node.value == self.problem.goal).all()
    
    def backup_path_from_node(self, node):
        path = [np.array(node.value)]
        while node.parent is not None:
            node = node.parent
            path.append(np.array(node.value))
        path.reverse()
        return path

def rrt_planning(problem, max_iterations=1000, prob_sample_q_goal=.05):
    """
    RRT* path planning
    Input: 
        problem (IiwaProblem): instance of a utility class
        max_iterations: the maximum number of samples to be collected 
        prob_sample_q_goal: the probability of sampling q_goal

    Output:
        path (list): [q_start, ...., q_goal]. 
                    Note q's are configurations, not RRT nodes 
    """

    rrt_tools = RRT_tools(problem)
    q_goal = problem.goal
    q_start = problem.start
    
    for k in range(max_iterations):
        q_sample = rrt_tools.sample_node_in_configuration_space()
        random_num = np.random.random_sample()
        if random_num < prob_sample_q_goal:
            q_sample = q_goal
        n_near = rrt_tools.find_nearest_node_in_RRT_graph(q_sample)
        q_new = rrt_tools.steer(n_near.value, q_sample)

        n_min = n_near
        c_min = rrt_tools.get_node_cost(n_min) + problem.cspace.distance(n_min.value, q_new)
        if q_new is not None:
            # connect along minimum cost path
            neighbors = rrt_tools.find_neighbors_in_RRT_graph(q_new)
            for (path_cost, n_neighbor) in neighbors: # sorted in best-first order
                if path_cost < c_min and rrt_tools.collision_free(n_neighbor.value, q_new):
                    n_min = n_neighbor
                    c_min = path_cost
                    break
            
            n_new = rrt_tools.grow_rrt_tree(n_min, q_new)
            root_to_new = rrt_tools.get_node_cost(n_new)

            # rewire tree
            for (path_cost, n_neighbor) in neighbors:
                new_neighbor_cost = root_to_new + problem.cspace.distance(n_new.value, n_neighbor.value)
                if rrt_tools.collision_free(n_neighbor.value, q_new) and new_neighbor_cost < n_neighbor.cost:
                    n_neighbor.parent = n_new
                    n_neighbor.cost = problem.cspace.distance(n_new.value, n_neighbor.value)

            if rrt_tools.node_reaches_goal(n_new):
                path = rrt_tools.backup_path_from_node(n_new)
                return path
    return None

In [ ]:
# Start, release, and end states (joint angles)

def find_windup_winddown(start_state, release_state_n, release_state_p, end_state):

    problem_start_to_release = IiwaProblem(start_state, release_state_n, wsg_position_open)
    for i in range(5):
        path_start_to_release = rrt_planning(problem_start_to_release)
        if path_start_to_release is not None:
            break

    # problem_release_to_end = IiwaProblem(release_state_p, end_state, wsg_position_open)

    # if path_start_to_release is not None:
    #     for i in range(5):
    #         path_release_to_end = rrt_planning(problem_release_to_end)
    #         if path_release_to_end is not None:
    #             break
    path_release_to_end = None
    
    return path_start_to_release, path_release_to_end

In [ ]:
theta_max_joints = np.array([2.96706, 2.0944, 2.96706, 2.0944, 2.96706, 2.0944, 3.05433])
theta_max_joints -= 0.01

'''
Plan for ramp up and ramp down:
- Draw a line along the velocity vector through the release configuration in joint space
- Find the farthest points on the line that do not violate joint limits and 
    are not in collision. (config_m and config_p)
- Do constant acceleration/decceleration (in joint space) from config_m to config_release and 
    config_release to config_p
'''

def find_throw_endpoints(q, v):
    alpha = 0.005
    v_dir = v/np.linalg.norm(v)

    config_m = q.copy()
    while (np.abs(config_m) < theta_max_joints).all() and not thrower.ExistsCollision(config_m, wsg_position_open):
        config_m -= alpha*v_dir
    config_m += alpha*v_dir # overcounted one

    config_p = q.copy()
    while (np.abs(config_p) < theta_max_joints).all() and not thrower.ExistsCollision(config_p, wsg_position_open):
        config_p += alpha*v_dir
    config_p -= alpha*v_dir # overcounted one

    return config_m, config_p

def time_parameterize_constant_a(v, config_m, config_release, config_p):
    times = []
    configs = []

    t_release = np.average(2*(config_release-config_m)/v)
    a_ramp = v/t_release

    ramp_steps = 5
    for i in range(ramp_steps):
        time = t_release/ramp_steps*i
        times.append(time)
        configs.append((a_ramp*time**2)/2+config_m)

    times.append(t_release)
    configs.append(config_release)

    t_end = np.average(2*(config_p-config_release)/v)
    a_down = -v/t_end

    for i in range(1, ramp_steps+1):
        time = t_end/ramp_steps*i
        times.append(t_release + time)
        configs.append((a_down*time**2)/2+v*time+config_release)

    times.append(100)
    configs.append(config_p)
    times.append(101)
    configs.append(config_p)
    times.append(102)
    configs.append(config_p)

    return np.array(times), np.array(configs), t_release, t_end + t_release

config_m, config_p = find_throw_endpoints(q,v)

times, configs, t_release, t_end = time_parameterize_constant_a(v, config_m, q, config_p)

print(t_release)
print(t_end)


print(f"q: {q}")
print(f"q_m: {config_m}")
print(f"q_p: {config_p}")

_, X_WG_m = get_iiwa_ee_gripper_pose(plant_f, config_m)
thrower.visualize_frame('release_m', X_WG_m, length=0.3)

_, X_WG_p = get_iiwa_ee_gripper_pose(plant_f, config_p)
thrower.visualize_frame('release_p', X_WG_p, length=0.3)

times_wsg = np.array([0, t_release, t_end])
pos_wsg = np.array([[wsg_position_close], [wsg_position_open], [wsg_position_open]])


[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09

In [ ]:
"""
Velocity Scaling, WSG trajectory
"""
# For gripper pose trajectory, we want PiecewisePolynomial.CubicWithContinuousSecondDerivatives()

path_start_to_release, path_release_to_end = find_windup_winddown(start_state, release_state_n, release_state_p, end_state)




def find_translational_path_length(path):
    """
        Total length of path, using this in joint space.
    """
    length = 0
    d_lengths = []
    for i in range(1, len(path)):
        _, X_WG_i_minus = get_iiwa_ee_gripper_pose(plant_f, np.array(path[i-1]))
        _, X_WG_i = get_iiwa_ee_gripper_pose(plant_f, np.array(path[i]))
        delta_translation = X_WG_i.translation() - X_WG_i_minus.translation()
        d_lengths.append(np.linalg.norm(delta_translation))
        length += np.linalg.norm(delta_translation)
    return length, d_lengths

def constant_accel(path, end_speed, time):
    """
        Make path in joint space be constant translational acceleration in cartesian
    """
    path = path
    accel = end_speed/time
    path_length, d_lengths = find_translational_path_length(path)
    times = [0]
    last_v = 0

    for i in range(1, len(path)):
        # p2 = (a*deltaT**2)/2 + v1*deltaT + p1
        # 0 = a*deltaT**2 + 2*v1*deltaT + 2*(p1-p2)

        dt = (-2*last_v + np.sqrt(4*last_v**2-8*accel*d_lengths[i-1]))/(2*accel)
        last_v = last_v + accel*dt
        times.append(times[i-1] + dt)

    return times, path


t_release = 5
end_speed = np.linalg.norm(v_G_release[3:6])

times, path = constant_accel(path_start_to_release, end_speed, t_release)

times.append(times[-1] + dt)
path.append(release_state)

times.append(times[-1] + dt)
path.append(release_state_p)

path = np.array(path)


In [ ]:
traj = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(times, configs.T)
wsg_traj = PiecewisePolynomial.ZeroOrderHold(times_wsg, pos_wsg.T)

# thrower = IIWA_Thrower(dartboard_pose, start_state=configs[0], traj=traj, wsg_traj=wsg_traj)

# thrower.throw(0.025, t_end+t)

In [ ]:
thrower = IIWA_Thrower(dartboard_pose, start_state=configs[0], traj=traj, wsg_traj=wsg_traj)

times, dart_poses, gripper_poses, iiwa_state = thrower.throw(0.02, t_end+t+0.5)

trial_name = "MISSED_4"

np.savez(trial_name + ".npz", times=times, dart_poses=dart_poses, gripper_poses=gripper_poses,
            iiwa_state=iiwa_state, q=q, v=v, X_WG_release=X_WG_release, v_G_release=v_G_release)

[-2.1263e-09  2.0000e-02 -1.4616e-08]
[-2.1263e-09  2.0000e-02 -1.4616e-08]
2.1228820926489984
2.122855520542367
2.122796906250381
2.1227287509773936
2.1226506816385826
2.122562332888098
2.122463346863834
2.122353373759481
2.1222320744853445
2.122099133445214
2.1219543273205494
2.1217975054676597
2.12162832179497
2.1214464479358726
2.121251608184185
2.121043509344878
2.120821858907974
2.1205863736358936
2.1203367804681226
2.1200728171743175
2.119794232757089
2.1195007864926425
2.1191922461287445
2.1188683848860066
2.118528977456812
2.118173794198471
2.1178025910922775
2.117415086792653
2.117010892949699
2.1165892238887047
2.1161476905055796
2.115681977796754
2.115199937206633
2.114699731178202
2.11418196380147
2.113646761917466
2.113094020632606
2.112523585159712
2.1119353022500365
2.11132902016448
2.1107045824417936
2.11006182556676
2.109400579511514
2.1087206697034064
2.108021919490933
2.1073367303144175
2.1066325597811293
2.105909013347618
2.1051660638471583
2.1044036224538996
2.103

In [ ]:
""" save trajectories, q and v """

filename = "HITS_BOARD"
i = 0
while exists(filename + str(i) + ".npz"):
    i += 1
np.savez(filename + str(i) + ".npz", path=configs, q=q, v=v)

In [ ]:
""" load trajectories, q and v """

filename = "HITS_BOARD0.npz"
loadednpz = np.load(filename)
(loaded_path, q, v) = (loadednpz["path"], loadednpz["q"], loadednpz["v"])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=865a3533-96c6-49e9-ab4c-53138efbbf27' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>